# 基于 MindQuantum 0.6.0 在半导体双量子点系统下实现 Grover 算法。

## 双量子比特 Grover。

In [2]:
from mindquantum import *

circ = Circuit()
circ += H.on(0)
circ += H.on(1)
circ += Z.on(1,0)
circ += H.on(0)
circ += H.on(1)
circ += Z.on(0)
circ += Z.on(1)
circ += Z.on(1,0)
circ += H.on(0)
circ += H.on(1)

sim = Simulator('projectq', 2)
sim.apply_circuit(circ)
print(sim.get_qs(ket=True))

1¦11⟩


In [29]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

def _matrix_(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_(coeff):
    return -1j*_matrix_(coeff)@(s_z*dt)

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*ddt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*ddt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10

cz_params = np.array([1.5472503,  1.4179231,  1.540713,   1.9724044,  1.9253408,  1.3879265,
                     0.8130467,  0.76446086, 1.2703444,  1.8553745,  1.0291328,  1.2492974,
                     0.7880994,  0.3026381,  0.31203356, 0.30834132, 0.9533752,  1.3802187,
                     1.270656,   0.5646567,  0.94619316, 0.97377133, 1.9658349,  0.83277696,
                     1.0190777,  0.90001523, 0.26008993, 0.16526282, 0.22249524, 1.1596956,
                     1.5285202,  0.4919534,  0.01645389, 0.02608137, 0.6504683,  0.31325826,
                     0.4486266,  0.8677286,  1.3571227,  1.4995408,  1.1248059,  0.5996333,
                     0.32797617, 0.54987127])

h_params = np.array([0.93665886, 0.24112344, 3.1801345,  1.1307619,  1.5094017,  3.022603,
                     0.45939285, 2.7383144,  1.6714032,  2.1061618,  2.3177178,  1.2582982])

z_params = np.array([1.6117827,  1.3258268,  0.0498501,  2.3190951,  0.27652496, 1.5626596,
                     0.70895815, 0.61659425, 1.8357016,  1.4761814,  2.117067,   0.6488876 ])

def cz_circ():
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[:10]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[10:20]])
    circ_ += Circuit([gate_c_0(cz_params[20]).on([1,0]), gate_c_0(cz_params[21]).on([1,0])])
    circ_ += Circuit([gate_c_1(cz_params[22]).on([1,0]), gate_c_1(cz_params[23]).on([1,0])])
    circ_ += Circuit([gate_0(param).on(0) for param in cz_params[24:34]])
    circ_ += Circuit([gate_0(0).on(1) for i in range(10,20)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(1) for param in cz_params[34:]])
    circ_ += BarrierGate()
    return circ_

def i_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(0).on(qubit)
    return circ_

def h_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(h_params[i]).on(qubit)
    return circ_

def z_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(z_params[i]).on(qubit)
    return circ_

circ = Circuit()

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += h_circ(1)

circ += cz_circ()

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += h_circ(1)

circ += z_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += z_circ(1)

circ += cz_circ()

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(0)
circ += h_circ(1)


sim = Simulator('projectq', 2)
sim.apply_circuit(circ)
state = sim.get_qs()
distribution = []
for i in range(len(state)):
    distribution.append((state[i].conj()*state[i]).real)
print(distribution)

[2.6125416398212854e-05, 3.064878890397258e-05, 3.449741938457886e-05, 0.9999087283753169]


## 三量子比特 Grover 算法

In [30]:
from mindquantum import *

def CCZ():
    circ = Circuit()
    # circ += H.on(2)
    circ += X.on(2,0)
    circ += dagger(Circuit(T.on(2)))
    circ += X.on(2,1)
    circ += T.on(2)
    circ += X.on(2,0)
    circ += T.on(0)
    circ += dagger(Circuit(T.on(2)))
    circ += X.on(2,1)
    circ += X.on(0,1)
    circ += T.on(2)
    circ += dagger(Circuit(T.on(0)))
    circ += T.on(1)
    # circ += H.on(2)
    circ += X.on(0,1)
    return circ


# ccz = CCZ()
# ccz.svg()
circ = Circuit()
circ += UN(H, 3)
circ += CCZ()
# circ += Z.on(2,[0,1])
circ.svg()
sim = Simulator('projectq', 3)
sim.apply_circuit(circ)
print(sim.get_qs(True))



√2/4¦000⟩
√2/4¦001⟩
√2/4¦010⟩
√2/4¦011⟩
√2/4¦100⟩
√2/4¦101⟩
√2/4¦110⟩
-√2/4¦111⟩


In [36]:
from mindquantum import *


def CCZ(obj_qubit, ctrl_qubit_0, ctrl_qubit_1): # ccz(2, [0,1])
    circ = Circuit()
    circ += X.on(obj_qubit,ctrl_qubit_0)
    circ += dagger(Circuit(T.on(obj_qubit)))
    circ += X.on(obj_qubit,ctrl_qubit_1)
    circ += T.on(obj_qubit)
    circ += X.on(obj_qubit,ctrl_qubit_0)
    circ += T.on(ctrl_qubit_0)
    circ += dagger(Circuit(T.on(obj_qubit)))
    circ += X.on(obj_qubit,ctrl_qubit_1)
    circ += X.on(ctrl_qubit_0,ctrl_qubit_1)
    circ += T.on(obj_qubit)
    circ += dagger(Circuit(T.on(ctrl_qubit_0)))
    circ += T.on(ctrl_qubit_1)
    circ += X.on(ctrl_qubit_0,ctrl_qubit_1)
    return circ


circ = Circuit()
circ += H.on(0)
circ += H.on(1)
circ += H.on(2)

# circ += Z.on(2,[1,0])
circ += CCZ(2, 1, 0)

circ += H.on(0)
circ += H.on(1)
circ += H.on(2)

circ += Z.on(0)
circ += Z.on(1)
circ += Z.on(2)

circ += Z.on(1,0)
circ += Z.on(2,0)
circ += Z.on(2,1)

# circ += Z.on(2,[1,0])
circ += CCZ(2, 1, 0)

circ += H.on(0)
circ += H.on(1)
circ += H.on(2)

# circ += Z.on(2,[1,0])
circ += CCZ(2, 1, 0)

circ += H.on(0)
circ += H.on(1)
circ += H.on(2)

circ += Z.on(0)
circ += Z.on(1)
circ += Z.on(2)

circ += Z.on(1,0)
circ += Z.on(2,0)
circ += Z.on(2,1)

# circ += Z.on(2,[1,0])
circ += CCZ(2, 1, 0)

circ += H.on(0)
circ += H.on(1)
circ += H.on(2)

sim = Simulator('projectq', 3)
sim.apply_circuit(circ)
state = sim.get_qs()
distribution = []
for i in range(len(state)):
    distribution.append((state[i].conj()*state[i]).real)
print(distribution)

[0.007812500000000009, 0.007812499999999971, 0.007812500000000009, 0.007812500000000009, 0.007812499999999998, 0.007812500000000002, 0.007812499999999998, 0.9453125000000014]


In [26]:
s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

def _matrix_(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_(coeff):
    return -1j*_matrix_(coeff)@(s_z*dt)

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*ddt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*ddt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10

# cz_params = np.array([1.5472503,  1.4179231,  1.540713,   1.9724044,  1.9253408,  1.3879265,
#                      0.8130467,  0.76446086, 1.2703444,  1.8553745,  1.0291328,  1.2492974,
#                      0.7880994,  0.3026381,  0.31203356, 0.30834132, 0.9533752,  1.3802187,
#                      1.270656,   0.5646567,  0.94619316, 0.97377133, 1.9658349,  0.83277696,
#                      1.0190777,  0.90001523, 0.26008993, 0.16526282, 0.22249524, 1.1596956,
#                      1.5285202,  0.4919534,  0.01645389, 0.02608137, 0.6504683,  0.31325826,
#                      0.4486266,  0.8677286,  1.3571227,  1.4995408,  1.1248059,  0.5996333,
#                      0.32797617, 0.54987127]) # error = 10-5

cz_params = np.array([1.5469222,  1.4163866,  1.5392585,  1.972247,   1.92602,    1.3877034,
                    0.8116179,  0.7628339,  1.2693673,  1.8558168,  1.0307236,  1.2505771,
                    0.7876434,  0.30035374, 0.30896303, 0.30590588, 0.95258,    1.3812971,
                    1.2727345,  0.56576526, 0.94235504, 0.9723476,  1.9667805,  0.8354815,
                    1.0185238,  0.8981452,  0.2593307,  0.16673481, 0.22587535, 1.1638049,
                    1.5322605,  0.4955098,  0.01972879, 0.02794218, 0.64640003, 0.3103799,
                    0.4482256,  0.8699769,  1.3611945,  1.5033041,  1.1262282,  0.59920496,
                    0.32729074, 0.54994607]) # error = 1.3743*10-6

# cx_params = np.array([2.3973486 , 1.360061  , 0.5219009 , 0.1817361 , 0.4168763 ,
#                     1.17235   , 1.808837  , 1.9328339 , 1.1321076 , 0.21697357,
#                     0.26678494, 0.18694156, 0.17362106, 1.8773022 , 2.3033705 ,
#                     0.4273502 , 0.19230893, 0.06170359, 1.0831344 , 2.707224  ,
#                     0.9448203 , 0.9738663 , 1.9660488 , 0.83349556, 0.06222223,
#                     0.03883766, 0.6891924 , 2.4698865 , 3.029343  , 0.63929975,
#                     0.10049819, 0.10288943, 0.30465302, 0.44671252, 1.5978051 ,
#                     0.80121493, 0.14243181, 0.07353207, 0.68680173, 1.7578927 ,
#                     2.406618  , 1.9629031 , 1.1089846 , 0.7566514 ]) # error = 10-5

cx_params = np.array([2.398063,   1.3611677,  0.5228268,  0.18214883, 0.416518,   1.1714737,
                    1.8080113,  1.9324119,  1.1320429,  0.21669449, 0.2670381,  0.1864032,
                    0.17237417, 1.8758354,  2.3015504,  0.42505047, 0.19033408, 0.06077161,
                    1.0835253,  2.7084877,  0.942298,   0.97240144, 1.9668409,  0.8354523,
                    0.06077522, 0.03603362, 0.6863459,  2.4681404 , 3.0299234,  0.6388336,
                    0.09712582, 0.0982787,  0.30029526, 0.44418356, 1.596421,   0.8014792,
                    0.1444568,  0.07676652, 0.68961644, 1.7588882,  2.4056408,  1.9623693,
                    1.1104989,  0.7586806]) # error = 1.6937*10-6

# h_params = np.array([0.93665886, 0.24112344, 3.1801345,  1.1307619,  1.5094017,  3.022603,
#                      0.45939285, 2.7383144,  1.6714032,  2.1061618,  2.3177178,  1.2582982]) # error = 10-5

h_params = np.array([2.0514407,  1.9363537,  0.35803944, 2.0369184,  0.23786525, 1.171406,
                    1.5800645,  2.2761526,  3.8131604,  1.6421343,  3.8060117,  2.5762186 ]) # error = 5.55*10-16

# z_params = np.array([1.6117827,  1.3258268,  0.0498501,  2.3190951,  0.27652496, 1.5626596,
#                      0.70895815, 0.61659425, 1.8357016,  1.4761814,  2.117067,   0.6488876 ]) # error = 10-5

z_params = np.array([1.6123164,  1.3263252,  0.0496451,  2.319243,   0.2757638,  1.5637369,
                    0.7095591,  0.61658764, 1.8363029,  1.4767408,  2.1175082,  0.6493106]) # error = 1.22 10^-15

# t_params = np.array([0.48441944, 1.8916097,  0.8680431,  1.7277799,  1.4107559,  0.4217147,
#                         0.12093126, 1.8459066,  0.1923866,  1.5161362,  1.2848788,  0.49415705]) # error = 10-5

t_params = np.array([0.47577828, 1.8931204,  0.86798567, 1.7298105,  1.4127659,  0.42006552,
                    0.12006928, 1.8473946,  0.1948133,  1.5128795,  1.2841252,  0.49527916]) # error = 3.89*10-15

# t_dagger_params = np.array([0.185471,   1.201903,   0.1577174,  1.3992891,  0.10488562, 0.08191212,
#                     1.969143,   1.3674749,  0.4603755,  2.485149,   0.59422135, 0.9907039 ]) # error = 10-5

t_dagger_params = np.array([0.18003628, 1.1424965,  3.4571714,  0.23706162, 1.8620081,  1.4052668,
                    2.877875,   1.9378989,  1.6281456,  0.10832042, 4.799379,   3.856666]) # error = 3.33*10-16


def cz_circ(obj_qubit, ctrl_qubit):
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[:10]])
    circ_ += Circuit([gate_0(ctrl_qubit).on(obj_qubit) for i in range(10)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(ctrl_qubit).on(ctrl_qubit) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[10:20]])
    circ_ += Circuit([gate_c_0(cz_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cz_params[21]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_c_1(cz_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cz_params[23]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[24:34]])
    circ_ += Circuit([gate_0(ctrl_qubit).on(obj_qubit) for i in range(10,20)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(ctrl_qubit).on(0) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[34:]])
    circ_ += BarrierGate()
    return circ_

circ = Circuit()
circ += X.on(0)
circ += X.on(1)
circ += cz_circ(1,0)
sim = Simulator('projectq', 2)
sim.apply_circuit(circ)
print(sim.get_qs(ket=True))
circ = Circuit()
circ += X.on(0)
circ += X.on(1)
circ += Z.on(1,0)
sim.reset()
sim.apply_circuit(circ)
print(sim.get_qs(ket=True))


[[ 8.93104768e-05+6.55320052e-04j  6.60583930e-04-3.11286781e-04j
   5.30538634e-04+1.32613065e-04j -3.33635297e-01-9.42701554e-01j]
 [-5.26095701e-05-5.25650665e-05j -1.48719652e-03+1.26326703e-03j
  -3.39922073e-01-9.40451419e-01j  3.25233743e-04-4.40205274e-04j]
 [ 2.37213009e-04-1.85820407e-04j -3.39563981e-01-9.40580603e-01j
  -1.95134590e-03-2.08349466e-05j  7.07773236e-04-1.79056614e-04j]
 [ 3.33401591e-01+9.42784623e-01j -3.01782842e-04+6.91352074e-06j
   6.80141077e-06-7.34642023e-05j  3.42705793e-04+5.65455243e-04j]]
(8.931047678254405e-05+0.0006553200521767886j)¦00⟩
(-5.26095700865703e-05-5.256506647147163e-05j)¦01⟩
(0.00023721300872215435-0.0001858204067014602j)¦10⟩
(0.33340159075024633+0.9427846230896992j)¦11⟩
-1¦11⟩


In [37]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

def _matrix_(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_(coeff):
    return -1j*_matrix_(coeff)@(s_z*dt)

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*ddt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*ddt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10

cz_params = np.array([1.5472503,  1.4179231,  1.540713,   1.9724044,  1.9253408,  1.3879265,
                     0.8130467,  0.76446086, 1.2703444,  1.8553745,  1.0291328,  1.2492974,
                     0.7880994,  0.3026381,  0.31203356, 0.30834132, 0.9533752,  1.3802187,
                     1.270656,   0.5646567,  0.94619316, 0.97377133, 1.9658349,  0.83277696,
                     1.0190777,  0.90001523, 0.26008993, 0.16526282, 0.22249524, 1.1596956,
                     1.5285202,  0.4919534,  0.01645389, 0.02608137, 0.6504683,  0.31325826,
                     0.4486266,  0.8677286,  1.3571227,  1.4995408,  1.1248059,  0.5996333,
                     0.32797617, 0.54987127])

cx_params = np.array([2.3973486 , 1.360061  , 0.5219009 , 0.1817361 , 0.4168763 ,
                    1.17235   , 1.808837  , 1.9328339 , 1.1321076 , 0.21697357,
                    0.26678494, 0.18694156, 0.17362106, 1.8773022 , 2.3033705 ,
                    0.4273502 , 0.19230893, 0.06170359, 1.0831344 , 2.707224  ,
                    0.9448203 , 0.9738663 , 1.9660488 , 0.83349556, 0.06222223,
                    0.03883766, 0.6891924 , 2.4698865 , 3.029343  , 0.63929975,
                    0.10049819, 0.10288943, 0.30465302, 0.44671252, 1.5978051 ,
                    0.80121493, 0.14243181, 0.07353207, 0.68680173, 1.7578927 ,
                    2.406618  , 1.9629031 , 1.1089846 , 0.7566514 ])

h_params = np.array([0.93665886, 0.24112344, 3.1801345,  1.1307619,  1.5094017,  3.022603,
                     0.45939285, 2.7383144,  1.6714032,  2.1061618,  2.3177178,  1.2582982])

z_params = np.array([1.6117827,  1.3258268,  0.0498501,  2.3190951,  0.27652496, 1.5626596,
                     0.70895815, 0.61659425, 1.8357016,  1.4761814,  2.117067,   0.6488876 ])

t_params = np.array([0.48441944, 1.8916097,  0.8680431,  1.7277799,  1.4107559,  0.4217147,
                        0.12093126, 1.8459066,  0.1923866,  1.5161362,  1.2848788,  0.49415705])

t_dagger_params = np.array([0.185471,   1.201903,   0.1577174,  1.3992891,  0.10488562, 0.08191212,
                    1.969143,   1.3674749,  0.4603755,  2.485149,   0.59422135, 0.9907039 ])



def cz_circ(obj_qubit, ctrl_qubit):
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[:10]])
    circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[10:20]])
    circ_ += Circuit([gate_c_0(cz_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cz_params[21]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_c_1(cz_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cz_params[23]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[24:34]])
    circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[34:]])
    circ_ += BarrierGate()
    return circ_

def cx_circ(obj_qubit, ctrl_qubit):
    circ_ = Circuit()
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cx_params[:10]])
    circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cx_params[10:20]])
    circ_ += Circuit([gate_c_0(cx_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cx_params[21]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_c_1(cx_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cx_params[23]).on([obj_qubit,ctrl_qubit])])
    circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cx_params[24:34]])
    circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
    circ_ += BarrierGate()
    circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
    circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cx_params[34:]])
    circ_ += BarrierGate()
    return circ_

def i_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(0).on(qubit)
    return circ_


def h_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(h_params[i]).on(qubit)
    return circ_

def z_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(z_params[i]).on(qubit)
    return circ_

def t_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(t_params[i]).on(qubit)
    return circ_

def t_dagger_circ(qubit=0):
    circ_ = Circuit()
    circ_ += BarrierGate()
    for i in range(12):
        circ_ += gate(t_dagger_params[i]).on(qubit)
    return circ_

def ccz_circ(obj_qubit, ctrl_qubit_0, ctrl_qubit_1):
    _circ = Circuit()
    
    _circ += cx_circ(obj_qubit, ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    
    _circ += t_dagger_circ(obj_qubit)
    _circ += i_circ(ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    
    _circ += cx_circ(obj_qubit, ctrl_qubit_1)
    _circ += i_circ(ctrl_qubit_0)
    
    _circ += t_circ(obj_qubit)
    _circ += i_circ(ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    
    _circ += cx_circ(obj_qubit, ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    
    _circ += t_circ(ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    _circ += i_circ(obj_qubit)
    
    _circ += t_dagger_circ(obj_qubit)
    _circ += i_circ(ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    
    _circ += cx_circ(obj_qubit, ctrl_qubit_1)
    _circ += i_circ(ctrl_qubit_0)
    
    _circ += cx_circ(ctrl_qubit_0, ctrl_qubit_1)
    _circ += i_circ(obj_qubit)
    
    _circ += t_circ(obj_qubit)
    _circ += i_circ(ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    
    _circ += t_dagger_circ(ctrl_qubit_0)
    _circ += i_circ(ctrl_qubit_1)
    _circ += i_circ(obj_qubit)
    
    _circ += t_circ(ctrl_qubit_1)
    _circ += i_circ(ctrl_qubit_0)
    _circ += i_circ(obj_qubit)
    
    _circ += cx_circ(ctrl_qubit_0, ctrl_qubit_1)
    _circ += i_circ(obj_qubit)
    
    return _circ
    

circ = Circuit()

# Hs
circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += h_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += i_circ(1)
circ += h_circ(2)

circ += ccz_circ(2, 1, 0)

# Hs
circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += h_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += i_circ(1)
circ += h_circ(2)

# Zs
circ += z_circ(0)
circ += i_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += z_circ(1) 
circ += i_circ(2)

circ += i_circ(0)
circ += i_circ(1)
circ += z_circ(2)

circ += cz_circ(1, 0)
circ += i_circ(2)

circ += cz_circ(2, 0)
circ += i_circ(1)

circ += cz_circ(2, 1)
circ += i_circ(0)

circ += ccz_circ(2, 1, 0)

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += h_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += i_circ(1)
circ += h_circ(2)

circ += ccz_circ(2, 1, 0)

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += h_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += i_circ(1)
circ += h_circ(2)

circ += z_circ(0)
circ += i_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += z_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += i_circ(1)
circ += z_circ(2)

circ += cz_circ(1, 0)
circ += i_circ(2)

circ += cz_circ(2, 0)
circ += i_circ(1)

circ += cz_circ(2, 1)
circ += i_circ(0)

circ += ccz_circ(2, 1, 0)

circ += h_circ(0)
circ += i_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += h_circ(1)
circ += i_circ(2)

circ += i_circ(0)
circ += i_circ(1)
circ += h_circ(2)


sim = Simulator('projectq', 3)
sim.apply_circuit(circ)
state = sim.get_qs()
distribution = []
for i in range(len(state)):
    distribution.append((state[i].conj()*state[i]).real)
print(distribution)

[0.00609863726903059, 0.00598196354797065, 0.005327819543016596, 0.006147099206253692, 0.004497858400822129, 0.004485004963751634, 0.003994228763469529, 0.9634673883056495]


# h 噪声对计算结果的影响

In [3]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

cz_params = np.array([1.5469222,  1.4163866,  1.5392585,  1.972247,   1.92602,    1.3877034,
                    0.8116179,  0.7628339,  1.2693673,  1.8558168,  1.0307236,  1.2505771,
                    0.7876434,  0.30035374, 0.30896303, 0.30590588, 0.95258,    1.3812971,
                    1.2727345,  0.56576526, 0.94235504, 0.9723476,  1.9667805,  0.8354815,
                    1.0185238,  0.8981452,  0.2593307,  0.16673481, 0.22587535, 1.1638049,
                    1.5322605,  0.4955098,  0.01972879, 0.02794218, 0.64640003, 0.3103799,
                    0.4482256,  0.8699769,  1.3611945,  1.5033041,  1.1262282,  0.59920496,
                    0.32729074, 0.54994607]) # error = 1.3743*10-6

cx_params = np.array([2.398063,   1.3611677,  0.5228268,  0.18214883, 0.416518,   1.1714737,
                    1.8080113,  1.9324119,  1.1320429,  0.21669449, 0.2670381,  0.1864032,
                    0.17237417, 1.8758354,  2.3015504,  0.42505047, 0.19033408, 0.06077161,
                    1.0835253,  2.7084877,  0.942298,   0.97240144, 1.9668409,  0.8354523,
                    0.06077522, 0.03603362, 0.6863459,  2.4681404 , 3.0299234,  0.6388336,
                    0.09712582, 0.0982787,  0.30029526, 0.44418356, 1.596421,   0.8014792,
                    0.1444568,  0.07676652, 0.68961644, 1.7588882,  2.4056408,  1.9623693,
                    1.1104989,  0.7586806]) # error = 1.6937*10-6

h_params = np.array([2.0514407,  1.9363537,  0.35803944, 2.0369184,  0.23786525, 1.171406,
                    1.5800645,  2.2761526,  3.8131604,  1.6421343,  3.8060117,  2.5762186 ]) # error = 5.55*10-16

z_params = np.array([1.6123164,  1.3263252,  0.0496451,  2.319243,   0.2757638,  1.5637369,
                    0.7095591,  0.61658764, 1.8363029,  1.4767408,  2.1175082,  0.6493106]) # error = 1.22 10^-15

t_params = np.array([0.47577828, 1.8931204,  0.86798567, 1.7298105,  1.4127659,  0.42006552,
                    0.12006928, 1.8473946,  0.1948133,  1.5128795,  1.2841252,  0.49527916]) # error = 3.89*10-15

t_dagger_params = np.array([0.18003628, 1.1424965,  3.4571714,  0.23706162, 1.8620081,  1.4052668,
                    2.877875,   1.9378989,  1.6281456,  0.10832042, 4.799379,   3.856666]) # error = 3.33*10-16

error_result = []

for dh in [-0.0010, -0.0009, -0.0008, -0.0007, -0.0006, -0.0005, -0.0004, -0.0003, -0.0002, -0.0001, -0.000,  0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.0010]:
    def _matrix_(coeff):
        return expm(-1j*(coeff*s_z+(1+dh)*s_x)*dt)

    def _diff_matrix_(coeff):
        return -1j*_matrix_(coeff)@(s_z*dt)

    def _matrix_0(coeff):
        return expm(-1j*(coeff*s_z+(1+dh)*s_x)*ddt)

    def _diff_matrix_0(coeff):
        return -1j*_matrix_0(coeff)@(s_z*ddt)

    def _matrix_c_0(coeff):
        return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron((1+dh)*s_x, one) + kron(one, (1+dh)*s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_0(coeff):
        return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

    def _matrix_c_1(coeff):
        return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron((1+dh)*s_x, one) + kron(one, (1+dh)*s_x) + coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_1(coeff):
        return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

    gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
    gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
    gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
    gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10


    def cz_circ(obj_qubit, ctrl_qubit):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[:10]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[10:20]])
        circ_ += Circuit([gate_c_0(cz_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cz_params[21]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_c_1(cz_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cz_params[23]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[24:34]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[34:]])
        circ_ += BarrierGate()
        return circ_

    def cx_circ(obj_qubit, ctrl_qubit):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cx_params[:10]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cx_params[10:20]])
        circ_ += Circuit([gate_c_0(cx_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cx_params[21]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_c_1(cx_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cx_params[23]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cx_params[24:34]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cx_params[34:]])
        circ_ += BarrierGate()
        return circ_

    def i_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(0).on(qubit)
        return circ_


    def h_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(h_params[i]).on(qubit)
        return circ_

    def z_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(z_params[i]).on(qubit)
        return circ_

    def t_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(t_params[i]).on(qubit)
        return circ_

    def t_dagger_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(t_dagger_params[i]).on(qubit)
        return circ_

    def ccz_circ(obj_qubit, ctrl_qubit_0, ctrl_qubit_1):
        _circ = Circuit()
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += t_dagger_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_1)
        _circ += i_circ(ctrl_qubit_0)
        
        _circ += t_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += t_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        _circ += t_dagger_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_1)
        _circ += i_circ(ctrl_qubit_0)
        
        _circ += cx_circ(ctrl_qubit_0, ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        _circ += t_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += t_dagger_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        _circ += t_circ(ctrl_qubit_1)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(obj_qubit)
        
        _circ += cx_circ(ctrl_qubit_0, ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        return _circ
        

    circ = Circuit()

    # Hs
    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    circ += ccz_circ(2, 1, 0)

    # Hs
    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    # Zs
    circ += z_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += z_circ(1) 
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += z_circ(2)

    circ += cz_circ(1, 0)
    circ += i_circ(2)

    circ += cz_circ(2, 0)
    circ += i_circ(1)

    circ += cz_circ(2, 1)
    circ += i_circ(0)

    circ += ccz_circ(2, 1, 0)

    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    circ += ccz_circ(2, 1, 0)

    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    circ += z_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += z_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += z_circ(2)

    circ += cz_circ(1, 0)
    circ += i_circ(2)

    circ += cz_circ(2, 0)
    circ += i_circ(1)

    circ += cz_circ(2, 1)
    circ += i_circ(0)

    circ += ccz_circ(2, 1, 0)

    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)


    sim = Simulator('projectq', 3)
    sim.apply_circuit(circ)
    state = sim.get_qs()
    distribution = []
    for i in range(len(state)):
        distribution.append((state[i].conj()*state[i]).real)
    print(dh, distribution[-1])
    error_result.append(distribution[-1])
    
print(error_result)
    


-0.001 0.6699514728732147
-0.0009 0.7126108187896742
-0.0008 0.7538689913520062
-0.0007 0.7929655042240465
-0.0006 0.8291361652224143
-0.0005 0.861630576006468
-0.0004 0.8897307006439334
-0.0003 0.9127699829036712
-0.0002 0.930152414925192
-0.0001 0.9413709019831191
-0.0 0.9460242345211193
0.0001 0.9438319736740957
0.0002 0.9346465831533544
0.0003 0.9184622001953576
0.0004 0.8954195310670603
0.0005 0.8658064803551961
0.0006 0.8300542739449225
0.0007 0.7887290074592838
0.0008 0.742518737505732
0.0009 0.692216423730969
0.001 0.6386992157447217
[0.6699514728732147, 0.7126108187896742, 0.7538689913520062, 0.7929655042240465, 0.8291361652224143, 0.861630576006468, 0.8897307006439334, 0.9127699829036712, 0.930152414925192, 0.9413709019831191, 0.9460242345211193, 0.9438319736740957, 0.9346465831533544, 0.9184622001953576, 0.8954195310670603, 0.8658064803551961, 0.8300542739449225, 0.7887290074592838, 0.742518737505732, 0.692216423730969, 0.6386992157447217]


# 讨论 J 误差对结果的影响

In [7]:
import copy
import numpy as np
import mindspore as ms
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
from mindspore.ops import operations
from mindspore import nn, ops, Tensor, context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
from mindspore.nn import Adam, TrainOneStepCell, LossBase
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
ms.set_seed(1)
np.random.seed(1)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/2
ddt = np.pi/10

cz_params = np.array([1.5469222,  1.4163866,  1.5392585,  1.972247,   1.92602,    1.3877034,
                    0.8116179,  0.7628339,  1.2693673,  1.8558168,  1.0307236,  1.2505771,
                    0.7876434,  0.30035374, 0.30896303, 0.30590588, 0.95258,    1.3812971,
                    1.2727345,  0.56576526, 0.94235504, 0.9723476,  1.9667805,  0.8354815,
                    1.0185238,  0.8981452,  0.2593307,  0.16673481, 0.22587535, 1.1638049,
                    1.5322605,  0.4955098,  0.01972879, 0.02794218, 0.64640003, 0.3103799,
                    0.4482256,  0.8699769,  1.3611945,  1.5033041,  1.1262282,  0.59920496,
                    0.32729074, 0.54994607]) # error = 1.3743*10-6

cx_params = np.array([2.398063,   1.3611677,  0.5228268,  0.18214883, 0.416518,   1.1714737,
                    1.8080113,  1.9324119,  1.1320429,  0.21669449, 0.2670381,  0.1864032,
                    0.17237417, 1.8758354,  2.3015504,  0.42505047, 0.19033408, 0.06077161,
                    1.0835253,  2.7084877,  0.942298,   0.97240144, 1.9668409,  0.8354523,
                    0.06077522, 0.03603362, 0.6863459,  2.4681404 , 3.0299234,  0.6388336,
                    0.09712582, 0.0982787,  0.30029526, 0.44418356, 1.596421,   0.8014792,
                    0.1444568,  0.07676652, 0.68961644, 1.7588882,  2.4056408,  1.9623693,
                    1.1104989,  0.7586806]) # error = 1.6937*10-6

h_params = np.array([2.0514407,  1.9363537,  0.35803944, 2.0369184,  0.23786525, 1.171406,
                    1.5800645,  2.2761526,  3.8131604,  1.6421343,  3.8060117,  2.5762186 ]) # error = 5.55*10-16

z_params = np.array([1.6123164,  1.3263252,  0.0496451,  2.319243,   0.2757638,  1.5637369,
                    0.7095591,  0.61658764, 1.8363029,  1.4767408,  2.1175082,  0.6493106]) # error = 1.22 10^-15

t_params = np.array([0.47577828, 1.8931204,  0.86798567, 1.7298105,  1.4127659,  0.42006552,
                    0.12006928, 1.8473946,  0.1948133,  1.5128795,  1.2841252,  0.49527916]) # error = 3.89*10-15

t_dagger_params = np.array([0.18003628, 1.1424965,  3.4571714,  0.23706162, 1.8620081,  1.4052668,
                    2.877875,   1.9378989,  1.6281456,  0.10832042, 4.799379,   3.856666]) # error = 3.33*10-16

error_result = []

for dj in [-0.0010, -0.0009, -0.0008, -0.0007, -0.0006, -0.0005, -0.0004, -0.0003, -0.0002, -0.0001, -0.000,  0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.0010]:
    def _matrix_(coeff):
        return expm(-1j*((1+dj)*coeff*s_z+s_x)*dt)

    def _diff_matrix_(coeff):
        return -1j*_matrix_((1+dj)*coeff)@(s_z*dt)

    def _matrix_0(coeff):
        return expm(-1j*((1+dj)*coeff*s_z+s_x)*ddt)

    def _diff_matrix_0(coeff):
        return -1j*_matrix_0((1+dj)*coeff)@(s_z*ddt)

    def _matrix_c_0(coeff):
        return expm(-1j*((1+dj)*coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + (1+dj)*coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_0(coeff):
        return -1j*_matrix_c_0((1+dj)*coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*ddt)

    def _matrix_c_1(coeff):
        return expm(-1j*(kron(s_z, one) + (1+dj)*coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + (1+dj)*coeff*kron(s_z-one, s_z-one))*5*ddt)

    def _diff_matrix_c_1(coeff):
        return -1j*_matrix_c_1((1+dj)*coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*ddt)

    gate = gene_univ_parameterized_gate('gete', _matrix_, _diff_matrix_) # dt=pi/2
    gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0) # ddt=pi/10
    gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0) # ddt=pi/10
    gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1) # ddt=pi/10


    def cz_circ(obj_qubit, ctrl_qubit):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[:10]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[10:20]])
        circ_ += Circuit([gate_c_0(cz_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cz_params[21]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_c_1(cz_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cz_params[23]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cz_params[24:34]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cz_params[34:]])
        circ_ += BarrierGate()
        return circ_

    def cx_circ(obj_qubit, ctrl_qubit):
        circ_ = Circuit()
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cx_params[:10]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cx_params[10:20]])
        circ_ += Circuit([gate_c_0(cx_params[20]).on([obj_qubit,ctrl_qubit]), gate_c_0(cx_params[21]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_c_1(cx_params[22]).on([obj_qubit,ctrl_qubit]), gate_c_1(cx_params[23]).on([obj_qubit,ctrl_qubit])])
        circ_ += Circuit([gate_0(param).on(ctrl_qubit) for param in cx_params[24:34]])
        circ_ += Circuit([gate_0(0).on(obj_qubit) for i in range(10,20)])
        circ_ += BarrierGate()
        circ_ += Circuit([gate_0(0).on(ctrl_qubit) for i in range(10)])
        circ_ += Circuit([gate_0(param).on(obj_qubit) for param in cx_params[34:]])
        circ_ += BarrierGate()
        return circ_

    def i_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(0).on(qubit)
        return circ_


    def h_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(h_params[i]).on(qubit)
        return circ_

    def z_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(z_params[i]).on(qubit)
        return circ_

    def t_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(t_params[i]).on(qubit)
        return circ_

    def t_dagger_circ(qubit=0):
        circ_ = Circuit()
        circ_ += BarrierGate()
        for i in range(12):
            circ_ += gate(t_dagger_params[i]).on(qubit)
        return circ_

    def ccz_circ(obj_qubit, ctrl_qubit_0, ctrl_qubit_1):
        _circ = Circuit()
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += t_dagger_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_1)
        _circ += i_circ(ctrl_qubit_0)
        
        _circ += t_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += t_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        _circ += t_dagger_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += cx_circ(obj_qubit, ctrl_qubit_1)
        _circ += i_circ(ctrl_qubit_0)
        
        _circ += cx_circ(ctrl_qubit_0, ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        _circ += t_circ(obj_qubit)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        
        _circ += t_dagger_circ(ctrl_qubit_0)
        _circ += i_circ(ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        _circ += t_circ(ctrl_qubit_1)
        _circ += i_circ(ctrl_qubit_0)
        _circ += i_circ(obj_qubit)
        
        _circ += cx_circ(ctrl_qubit_0, ctrl_qubit_1)
        _circ += i_circ(obj_qubit)
        
        return _circ
        

    circ = Circuit()

    # Hs
    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    circ += ccz_circ(2, 1, 0)

    # Hs
    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    # Zs
    circ += z_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += z_circ(1) 
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += z_circ(2)

    circ += cz_circ(1, 0)
    circ += i_circ(2)

    circ += cz_circ(2, 0)
    circ += i_circ(1)

    circ += cz_circ(2, 1)
    circ += i_circ(0)

    circ += ccz_circ(2, 1, 0)

    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    circ += ccz_circ(2, 1, 0)

    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)

    circ += z_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += z_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += z_circ(2)

    circ += cz_circ(1, 0)
    circ += i_circ(2)

    circ += cz_circ(2, 0)
    circ += i_circ(1)

    circ += cz_circ(2, 1)
    circ += i_circ(0)

    circ += ccz_circ(2, 1, 0)

    circ += h_circ(0)
    circ += i_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += h_circ(1)
    circ += i_circ(2)

    circ += i_circ(0)
    circ += i_circ(1)
    circ += h_circ(2)


    sim = Simulator('projectq', 3)
    sim.apply_circuit(circ)
    state = sim.get_qs()
    distribution = []
    for i in range(len(state)):
        distribution.append((state[i].conj()*state[i]).real)
    print(dj, distribution[-1])
    error_result.append(distribution[-1])
    
print(error_result)


-0.001 0.9037224733105496
-0.0009 0.9121911037225263
-0.0008 0.9197719363608083
-0.0007 0.9264400936581969
-0.0006 0.9321732150307719
-0.0005 0.9369515576905334
-0.0004 0.940758086737804
-0.0003 0.9435785540542503
-0.0002 0.9454015655711132
-0.0001 0.9462186365429188
-0.0 0.9460242345211193
0.0001 0.9448158097798107
0.0002 0.9425938130112717
0.0003 0.9393617001746257
0.0004 0.9351259244437444
0.0005 0.9298959152691346
0.0006 0.9236840446345475
0.0007 0.9165055806532891
0.0008 0.9083786287165758
0.0009 0.899324060470748
0.001 0.8893654309610467
[0.9037224733105496, 0.9121911037225263, 0.9197719363608083, 0.9264400936581969, 0.9321732150307719, 0.9369515576905334, 0.940758086737804, 0.9435785540542503, 0.9454015655711132, 0.9462186365429188, 0.9460242345211193, 0.9448158097798107, 0.9425938130112717, 0.9393617001746257, 0.9351259244437444, 0.9298959152691346, 0.9236840446345475, 0.9165055806532891, 0.9083786287165758, 0.899324060470748, 0.8893654309610467]
